In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import sobel
import time
from IPython.display import display, HTML
import warnings
import os
from pathlib import Path
from tqdm.notebook import tqdm
import glob

In [2]:
warnings.filterwarnings('ignore')

# Set better plotting defaults for Jupyter
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['figure.dpi'] = 100

In [ ]:
class TAPLAImageEnhancement:
    """
    Threshold-based Adaptive Power-Law Applications (TAPLA) 
    for Image Enhancement
    """
    
    def __init__(self, c=1.0, gamma=1.5, k=2.0, k1=0.1, window_size=11):
        """
        Parameters:
        - c: brightness control parameter
        - gamma: contrast control parameter  
        - k: sharpness/smoothness control parameter
        - k1: threshold adaptation control [0,1]
        - window_size: local window size (must be odd)
        """
        self.c = c
        self.gamma = gamma
        self.k = k
        self.k1 = k1
        self.w = window_size if window_size % 2 == 1 else window_size + 1
        
    def compute_integral_average_image(self, img):
        """
        Compute integral average image - Equations (3-5)
        """
        m, n = img.shape
        g = img.astype(np.float64).copy()
        
        # First row - Equation (3)
        for y in range(1, n):
            g[0, y] = (g[0, y-1] * y + g[0, y]) / (y + 1)
        
        # First column - Equation (4)
        for x in range(1, m):
            g[x, 0] = (g[x-1, 0] * x + g[x, 0]) / (x + 1)
        
        # Other points - Equation (5)
        for x in range(1, m):
            for y in range(1, n):
                r, c_idx = x - 1, y - 1
                g[x, y] = (g[x, y] + 
                          g[r, y] * r * (y + 1) + 
                          g[x, c_idx] * c_idx * (x + 1) - 
                          g[r, c_idx] * r * c_idx) / ((x + 1) * (y + 1))
        
        return g
    
    def compute_local_mean(self, integral_avg_img, x, y, m, n):
        """
        Compute local mean using integral average - Equations (6-7)
        """
        d = (self.w - 1) // 2
        
        x1 = max(0, x - d - 1)
        y1 = max(0, y - d - 1)
        x2 = min(m - 1, x + d)
        y2 = min(n - 1, y + d)
        
        s = (integral_avg_img[x2, y2] * (x2 + 1) * (y2 + 1))
        
        if x1 > 0 and y1 > 0:
            s += (integral_avg_img[x1, y1] * (x1 + 1) * (y1 + 1))
        
        if x1 > 0:
            s -= (integral_avg_img[x1, y2] * (x1 + 1) * (y2 + 1))
        
        if y1 > 0:
            s -= (integral_avg_img[x2, y1] * (x2 + 1) * (y1 + 1))
        
        actual_w = (x2 - x1) * (y2 - y1)
        local_mean = s / max(actual_w, 1)
        
        return local_mean
    
    def compute_threshold(self, intensity, local_mean):
        """
        Compute local threshold - Equation (9)
        """
        delta = abs(intensity - local_mean)
        denominator = max(1 - delta, 1e-10)
        threshold = local_mean * (1 + self.k1 * (delta / denominator) - 1)
        return threshold
    
    def tapla_transform(self, img):
        """
        Apply TAPLA transformation - Equations (12) and (15)
        """
        img_norm = img.astype(np.float64) / 255.0
        m, n = img_norm.shape
        
        integral_avg = self.compute_integral_average_image(img_norm)
        output = np.zeros_like(img_norm)
        
        for x in range(m):
            for y in range(n):
                r = img_norm[x, y]
                local_mean = self.compute_local_mean(integral_avg, x, y, m, n)
                T = self.compute_threshold(r, local_mean)
                d = r - T
                
                power = self.gamma * (1 - self.k * d)
                coefficient = self.c * (1 + self.k * d)
                
                if r > 0:
                    s = coefficient * (r ** power)
                else:
                    s = 0
                
                output[x, y] = np.clip(s, 0, 1)
        
        return (output * 255).astype(np.uint8)
    
    def calculate_eme(self, img, block_size=8):
        """
        Calculate EME - Equation (16)
        """
        m, n = img.shape
        p = m // block_size
        q = n // block_size
        
        eme = 0
        e = 1e-10
        
        for i in range(p):
            for j in range(q):
                block = img[i*block_size:(i+1)*block_size, 
                           j*block_size:(j+1)*block_size]
                
                I_max = np.max(block) + e
                I_min = np.min(block) + e
                
                if I_max > 0:
                    eme += np.log(I_max / I_min)
        
        return eme / (p * q)
    
    def calculate_tenengrad(self, img):
        """
        Calculate Tenengrad - Equations (17-18)
        """
        Gx = sobel(img.astype(np.float64), axis=1)
        Gy = sobel(img.astype(np.float64), axis=0)
        s = np.sqrt(Gx**2 + Gy**2)
        return np.sum(s**2)
    
    def enhance_image(self, img):
        """
        Main enhancement method
        Supports both grayscale and color images
        For color images, each channel (R, G, B) is processed separately
        """
        if len(img.shape) == 3:
            # Color image - process each channel separately
            print("🎨 Processing color image (R, G, B channels separately)...")
            output = np.zeros_like(img)
            for c in range(3):
                print(f"   Channel {['R', 'G', 'B'][c]}...", end=' ')
                output[:, :, c] = self.tapla_transform(img[:, :, c])
                print("✓")
            return output
        else:
            # Grayscale image
            print("⬜ Processing grayscale image...")
            return self.tapla_transform(img)


def upload_image_jupyter():
    """
    Upload image in Jupyter Notebook
    """
    try:
        # Try Google Colab first
        from google.colab import files
        print("📤 Click 'Choose Files' to upload your image...")
        uploaded = files.upload()
        filename = list(uploaded.keys())[0]
        print(f"✓ Uploaded: {filename}")
        return filename
    except:
        # For regular Jupyter
        print("📁 Please use: img = load_image('path/to/your/image.jpg')")
        print("   Or place your image in the same folder and use: img = load_image('image.jpg')")
        return None


def load_image(path, color=False):
    """
    Load image from path
    
    Parameters:
        path: image file path
        color: True for color image, False for grayscale
    
    Examples:
        img = load_image('image.jpg')  # Grayscale
        img = load_image('image.jpg', color=True)  # Color (RGB)
    """
    if color:
        img = cv2.imread(path)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            print(f"✓ Color image loaded: {img.shape}")
    else:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            print(f"✓ Grayscale image loaded: {img.shape}")
    
    if img is None:
        raise ValueError(f"❌ Could not load image from: {path}\n"
                        f"   Check if:\n"
                        f"   1. File exists at that location\n"
                        f"   2. Path is correct (use forward slashes /)\n"
                        f"   3. File format is supported (jpg, png, bmp, etc.)")
    
    return img


def batch_enhance_images(input_folder, output_folder, c=1.0, gamma=1.5, k=2.0, 
                         k1=0.1, window_size=11, color=False, 
                         file_extensions=['*.jpg', '*.jpeg', '*.png', '*.bmp'],
                         save_metrics=True):
    """
    Batch process multiple images from a folder
    
    Parameters:
        input_folder: path to folder containing input images
        output_folder: path to save enhanced images
        c, gamma, k, k1, window_size: TAPLA parameters
        color: True for color images, False for grayscale
        file_extensions: list of file patterns to process
        save_metrics: save metrics to CSV file
    
    Returns:
        results: dictionary with processing statistics
    
    Example:
        results = batch_enhance_images(
            input_folder='./input_images',
            output_folder='./enhanced_images',
            c=1.5, gamma=0.56, k=2.0, k1=0.2,
            color=True
        )
    """
    
    # Create output folder if not exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Get all image files
    image_files = []
    for ext in file_extensions:
        image_files.extend(glob.glob(os.path.join(input_folder, ext)))
        image_files.extend(glob.glob(os.path.join(input_folder, ext.upper())))
    
    if len(image_files) == 0:
        print(f"❌ No images found in {input_folder}")
        print(f"   Looking for: {file_extensions}")
        return None
    
    print("="*70)
    print("BATCH TAPLA IMAGE ENHANCEMENT")
    print("="*70)
    print(f"\n📁 Input folder:  {input_folder}")
    print(f"📁 Output folder: {output_folder}")
    print(f"🖼️  Found {len(image_files)} images")
    print(f"\n⚙️  Parameters: c={c}, γ={gamma}, k={k}, k₁={k1}, w={window_size}")
    print(f"🎨 Mode: {'Color (RGB)' if color else 'Grayscale'}")
    print("\n" + "="*70)
    
    # Initialize TAPLA
    tapla = TAPLAImageEnhancement(c, gamma, k, k1, window_size)
    
    # Statistics
    results = {
        'total': len(image_files),
        'success': 0,
        'failed': 0,
        'total_time': 0,
        'metrics': [],
        'failed_files': []
    }
    
    # Process each image with progress bar
    for img_path in tqdm(image_files, desc="Processing images"):
        try:
            filename = os.path.basename(img_path)
            
            # Load image
            if color:
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            else:
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            if img is None:
                results['failed'] += 1
                results['failed_files'].append(filename)
                continue
            
            # Calculate original metrics
            if color:
                img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            else:
                img_gray = img
            
            orig_eme = tapla.calculate_eme(img_gray)
            orig_ten = tapla.calculate_tenengrad(img_gray)
            
            # Enhance
            start_time = time.time()
            enhanced = tapla.enhance_image(img)
            elapsed_time = time.time() - start_time
            
            # Calculate enhanced metrics
            if color:
                enh_gray = cv2.cvtColor(enhanced, cv2.COLOR_RGB2GRAY)
            else:
                enh_gray = enhanced
            
            enh_eme = tapla.calculate_eme(enh_gray)
            enh_ten = tapla.calculate_tenengrad(enh_gray)
            
            # Save enhanced image
            output_path = os.path.join(output_folder, filename)
            if color:
                enhanced_bgr = cv2.cvtColor(enhanced, cv2.COLOR_RGB2BGR)
                cv2.imwrite(output_path, enhanced_bgr)
            else:
                cv2.imwrite(output_path, enhanced)
            
            # Store metrics
            if save_metrics:
                results['metrics'].append({
                    'filename': filename,
                    'orig_eme': orig_eme,
                    'enh_eme': enh_eme,
                    'eme_gain': ((enh_eme - orig_eme) / orig_eme * 100) if orig_eme > 0 else 0,
                    'orig_ten': orig_ten,
                    'enh_ten': enh_ten,
                    'ten_gain': ((enh_ten - orig_ten) / orig_ten * 100) if orig_ten > 0 else 0,
                    'time': elapsed_time
                })
            
            results['success'] += 1
            results['total_time'] += elapsed_time
            
        except Exception as e:
            results['failed'] += 1
            results['failed_files'].append(f"{filename} ({str(e)})")
    
    # Save metrics to CSV
    if save_metrics and len(results['metrics']) > 0:
        import pandas as pd
        df = pd.DataFrame(results['metrics'])
        csv_path = os.path.join(output_folder, 'enhancement_metrics.csv')
        df.to_csv(csv_path, index=False)
        
        # Save summary statistics
        summary = {
            'Total Images': results['total'],
            'Successfully Processed': results['success'],
            'Failed': results['failed'],
            'Total Time (s)': f"{results['total_time']:.2f}",
            'Average Time per Image (s)': f"{results['total_time']/results['success']:.3f}" if results['success'] > 0 else 0,
            'Average EME Gain (%)': f"{df['eme_gain'].mean():.2f}",
            'Average TEN Gain (%)': f"{df['ten_gain'].mean():.2f}"
        }
        
        summary_df = pd.DataFrame([summary])
        summary_path = os.path.join(output_folder, 'summary.csv')
        summary_df.to_csv(summary_path, index=False)
    
    # Print summary
    print("\n" + "="*70)
    print("PROCESSING COMPLETE")
    print("="*70)
    print(f"✅ Successfully processed: {results['success']}/{results['total']} images")
    print(f"❌ Failed: {results['failed']} images")
    print(f"⏱️  Total time: {results['total_time']:.2f}s")
    if results['success'] > 0:
        print(f"⏱️  Average time per image: {results['total_time']/results['success']:.3f}s")
    
    if save_metrics and results['success'] > 0:
        print(f"\n📊 Metrics saved to:")
        print(f"   - {csv_path}")
        print(f"   - {summary_path}")
        
        # Print statistics
        import pandas as pd
        df = pd.DataFrame(results['metrics'])
        print(f"\n📈 Enhancement Statistics:")
        print(f"   Average EME gain: {df['eme_gain'].mean():.2f}%")
        print(f"   Average TEN gain: {df['ten_gain'].mean():.2f}%")
    
    if results['failed'] > 0:
        print(f"\n⚠️  Failed files:")
        for f in results['failed_files'][:10]:  # Show first 10
            print(f"   - {f}")
        if len(results['failed_files']) > 10:
            print(f"   ... and {len(results['failed_files'])-10} more")
    
    print("="*70)
    
    return results


def preview_batch_results(output_folder, num_samples=6):
    """
    Preview random samples from batch processing results
    
    Parameters:
        output_folder: folder containing enhanced images
        num_samples: number of random images to display
    """
    # Get all images in output folder
    image_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png', '*.bmp']:
        image_files.extend(glob.glob(os.path.join(output_folder, ext)))
    
    if len(image_files) == 0:
        print(f"❌ No images found in {output_folder}")
        return
    
    # Random sample
    num_samples = min(num_samples, len(image_files))
    sample_files = np.random.choice(image_files, num_samples, replace=False)
    
    # Display
    cols = 3
    rows = (num_samples + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))
    axes = axes.flatten() if num_samples > 1 else [axes]
    
    for idx, img_path in enumerate(sample_files):
        img = cv2.imread(img_path)
        if img is not None:
            if len(img.shape) == 3:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                axes[idx].imshow(img)
            else:
                axes[idx].imshow(img, cmap='gray')
            
            axes[idx].set_title(os.path.basename(img_path), fontsize=10)
            axes[idx].axis('off')
    
    # Hide extra subplots
    for idx in range(num_samples, len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()


def compare_batch_results(input_folder, output_folder, num_samples=3):
    """
    Compare original vs enhanced images side by side
    
    Parameters:
        input_folder: folder with original images
        output_folder: folder with enhanced images
        num_samples: number of comparisons to show
    """
    # Get common files
    input_files = set([os.path.basename(f) for f in glob.glob(os.path.join(input_folder, '*.*'))])
    output_files = set([os.path.basename(f) for f in glob.glob(os.path.join(output_folder, '*.*'))])
    
    # Exclude CSV files
    common_files = list(input_files.intersection(output_files))
    common_files = [f for f in common_files if not f.endswith('.csv')]
    
    if len(common_files) == 0:
        print("❌ No matching files found")
        return
    
    # Random sample
    num_samples = min(num_samples, len(common_files))
    sample_files = np.random.choice(common_files, num_samples, replace=False)
    
    # Display comparisons
    fig, axes = plt.subplots(num_samples, 2, figsize=(12, 4*num_samples))
    if num_samples == 1:
        axes = axes.reshape(1, -1)
    
    for idx, filename in enumerate(sample_files):
        # Load original
        orig_path = os.path.join(input_folder, filename)
        orig = cv2.imread(orig_path)
        if orig is not None and len(orig.shape) == 3:
            orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
        
        # Load enhanced
        enh_path = os.path.join(output_folder, filename)
        enh = cv2.imread(enh_path)
        if enh is not None and len(enh.shape) == 3:
            enh = cv2.cvtColor(enh, cv2.COLOR_BGR2RGB)
        
        # Display
        cmap = None if len(orig.shape) == 3 else 'gray'
        
        axes[idx, 0].imshow(orig, cmap=cmap)
        axes[idx, 0].set_title(f'Original: {filename}', fontsize=10)
        axes[idx, 0].axis('off')
        
        axes[idx, 1].imshow(enh, cmap=cmap)
        axes[idx, 1].set_title(f'Enhanced: {filename}', fontsize=10)
        axes[idx, 1].axis('off')
    
    plt.tight_layout()
    plt.show()


def create_sample_dataset(output_folder, num_images=10):
    """
    Create sample dataset for testing batch processing
    
    Parameters:
        output_folder: folder to save sample images
        num_images: number of sample images to create
    """
    os.makedirs(output_folder, exist_ok=True)
    
    print(f"Creating {num_images} sample images in {output_folder}...")
    
    for i in range(num_images):
        # Create different types of test images
        if i % 4 == 0:
            # Low contrast
            img = np.random.randint(80, 120, (256, 256, 3), dtype=np.uint8)
        elif i % 4 == 1:
            # Dark image
            img = np.random.randint(0, 80, (256, 256, 3), dtype=np.uint8)
        elif i % 4 == 2:
            # Gradient
            x = np.linspace(0, 255, 256)
            y = np.linspace(0, 255, 256)
            X, Y = np.meshgrid(x, y)
            img = np.stack([X, Y, (X+Y)/2], axis=2).astype(np.uint8)
        else:
            # Random
            img = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
        
        # Save
        filename = f"sample_{i+1:04d}.jpg"
        cv2.imwrite(os.path.join(output_folder, filename), img)
    
    print(f"✓ Created {num_images} sample images")
    return output_folder
    """
    Create sample test images
    """
    images = {}
    
    # Low contrast image
    images['low_contrast'] = np.random.randint(80, 120, (256, 256), dtype=np.uint8)
    
    # Dark image
    dark_img = np.random.randint(0, 80, (256, 256), dtype=np.uint8)
    images['dark'] = dark_img
    
    # Image with gradient
    x = np.linspace(0, 255, 256)
    y = np.linspace(0, 255, 256)
    X, Y = np.meshgrid(x, y)
    images['gradient'] = ((X + Y) / 2).astype(np.uint8)
    
    # Checkerboard pattern
    checker = np.zeros((256, 256), dtype=np.uint8)
    checker[::32, ::32] = 200
    checker[16::32, 16::32] = 200
    images['checker'] = checker
    
    return images


def plot_comparison(original, enhanced, params, metrics):
    """
    Plot original vs enhanced with metrics
    Supports both grayscale and color images
    """
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    is_color = len(original.shape) == 3
    cmap = None if is_color else 'gray'
    
    # Original image
    axes[0, 0].imshow(original, cmap=cmap)
    axes[0, 0].set_title('Original Image', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')
    
    # Enhanced image
    axes[0, 1].imshow(enhanced, cmap=cmap)
    axes[0, 1].set_title('TAPLA Enhanced', fontsize=12, fontweight='bold')
    axes[0, 1].axis('off')
    
    # Difference
    diff = cv2.absdiff(original, enhanced)
    if is_color:
        # For color images, show difference as color
        axes[0, 2].imshow(diff)
    else:
        # For grayscale, show as heatmap
        im = axes[0, 2].imshow(diff, cmap='hot')
        plt.colorbar(im, ax=axes[0, 2], fraction=0.046)
    axes[0, 2].set_title('Absolute Difference', fontsize=12, fontweight='bold')
    axes[0, 2].axis('off')
    
    # Histograms
    if is_color:
        # Color histograms for each channel
        colors = ('r', 'g', 'b')
        for i, col in enumerate(colors):
            axes[1, 0].hist(original[:, :, i].ravel(), 256, [0, 256], 
                           color=col, alpha=0.5, label=col.upper())
            axes[1, 1].hist(enhanced[:, :, i].ravel(), 256, [0, 256], 
                           color=col, alpha=0.5, label=col.upper())
        axes[1, 0].legend()
        axes[1, 1].legend()
    else:
        # Grayscale histograms
        axes[1, 0].hist(original.ravel(), 256, [0, 256], color='blue', alpha=0.7)
        axes[1, 1].hist(enhanced.ravel(), 256, [0, 256], color='green', alpha=0.7)
    
    axes[1, 0].set_title('Original Histogram', fontsize=12)
    axes[1, 0].set_xlabel('Pixel Intensity')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].grid(True, alpha=0.3)
    
    axes[1, 1].set_title('Enhanced Histogram', fontsize=12)
    axes[1, 1].set_xlabel('Pixel Intensity')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].grid(True, alpha=0.3)
    
    # Metrics text
    axes[1, 2].axis('off')
    metrics_text = f"""
    PARAMETERS:
    ──────────────────
    c (brightness): {params['c']}
    γ (contrast): {params['gamma']}
    k (sharpness): {params['k']}
    k₁ (threshold): {params['k1']}
    window size: {params['window_size']}
    
    METRICS:
    ──────────────────
    Original EME: {metrics['orig_eme']:.2f}
    Enhanced EME: {metrics['enh_eme']:.2f}
    EME Gain: {metrics['eme_gain']:.2f}%
    
    Original TEN: {metrics['orig_ten']:.0f}
    Enhanced TEN: {metrics['enh_ten']:.0f}
    TEN Gain: {metrics['ten_gain']:.2f}%
    
    Time: {metrics['time']:.3f}s
    """
    axes[1, 2].text(0.1, 0.5, metrics_text, fontsize=11, 
                    verticalalignment='center', fontfamily='monospace',
                    bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()


def run_tapla_experiment(img, c=1.0, gamma=1.5, k=2.0, k1=0.1, window_size=11):
    """
    Run single TAPLA experiment
    Supports both grayscale and color images
    """
    print("="*60)
    print("TAPLA IMAGE ENHANCEMENT EXPERIMENT")
    print("="*60)
    
    params = {
        'c': c,
        'gamma': gamma,
        'k': k,
        'k1': k1,
        'window_size': window_size
    }
    
    is_color = len(img.shape) == 3
    img_type = "Color (RGB)" if is_color else "Grayscale"
    
    print(f"\nImage size: {img.shape} ({img_type})")
    print(f"Parameters: c={c}, γ={gamma}, k={k}, k₁={k1}, w={window_size}")
    print("\nProcessing...")
    
    # Create TAPLA instance
    tapla = TAPLAImageEnhancement(**params)
    
    # Calculate original metrics (convert to grayscale for metrics if color)
    if is_color:
        img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        img_gray = img
    
    orig_eme = tapla.calculate_eme(img_gray)
    orig_ten = tapla.calculate_tenengrad(img_gray)
    
    # Enhance image
    start_time = time.time()
    enhanced = tapla.enhance_image(img)
    elapsed_time = time.time() - start_time
    
    # Calculate enhanced metrics
    if is_color:
        enhanced_gray = cv2.cvtColor(enhanced, cv2.COLOR_RGB2GRAY)
    else:
        enhanced_gray = enhanced
    
    enh_eme = tapla.calculate_eme(enhanced_gray)
    enh_ten = tapla.calculate_tenengrad(enhanced_gray)
    
    # Calculate gains
    eme_gain = ((enh_eme - orig_eme) / orig_eme) * 100 if orig_eme > 0 else 0
    ten_gain = ((enh_ten - orig_ten) / orig_ten) * 100 if orig_ten > 0 else 0
    
    metrics = {
        'orig_eme': orig_eme,
        'enh_eme': enh_eme,
        'eme_gain': eme_gain,
        'orig_ten': orig_ten,
        'enh_ten': enh_ten,
        'ten_gain': ten_gain,
        'time': elapsed_time
    }
    
    print(f"\n✓ Enhancement completed in {elapsed_time:.3f}s")
    print(f"  EME: {orig_eme:.2f} → {enh_eme:.2f} ({eme_gain:+.1f}%)")
    print(f"  TEN: {orig_ten:.0f} → {enh_ten:.0f} ({ten_gain:+.1f}%)")
    
    # Plot results
    plot_comparison(img, enhanced, params, metrics)
    
    return enhanced, metrics


def compare_multiple_params(img, params_list):
    """
    Compare multiple parameter sets
    """
    n_params = len(params_list)
    results = []
    
    print("="*60)
    print(f"COMPARING {n_params} DIFFERENT PARAMETER SETS")
    print("="*60)
    
    # Calculate original metrics once
    tapla = TAPLAImageEnhancement()
    orig_eme = tapla.calculate_eme(img)
    orig_ten = tapla.calculate_tenengrad(img)
    
    for i, params in enumerate(params_list):
        print(f"\n[{i+1}/{n_params}] Processing with: c={params['c']}, γ={params['gamma']}, "
              f"k={params['k']}, k₁={params['k1']}")
        
        tapla = TAPLAImageEnhancement(**params)
        
        start_time = time.time()
        enhanced = tapla.enhance_image(img)
        elapsed_time = time.time() - start_time
        
        enh_eme = tapla.calculate_eme(enhanced)
        enh_ten = tapla.calculate_tenengrad(enhanced)
        
        results.append({
            'params': params,
            'enhanced': enhanced,
            'eme': enh_eme,
            'ten': enh_ten,
            'time': elapsed_time
        })
        
        print(f"  Time: {elapsed_time:.3f}s | EME: {enh_eme:.2f} | TEN: {enh_ten:.0f}")
    
    # Visualization
    fig = plt.figure(figsize=(18, 10))
    
    # Original image
    ax = plt.subplot(3, n_params + 1, 1)
    ax.imshow(img, cmap='gray')
    ax.set_title(f'Original\nEME: {orig_eme:.1f}\nTEN: {orig_ten:.0f}', 
                 fontsize=10, fontweight='bold')
    ax.axis('off')
    
    ax = plt.subplot(3, n_params + 1, n_params + 2)
    ax.hist(img.ravel(), 256, [0, 256], color='gray', alpha=0.7)
    ax.set_title('Original Histogram', fontsize=9)
    ax.grid(True, alpha=0.3)
    
    ax = plt.subplot(3, n_params + 1, 2*n_params + 3)
    ax.axis('off')
    
    # Enhanced images
    for i, result in enumerate(results):
        # Image
        ax = plt.subplot(3, n_params + 1, i + 2)
        ax.imshow(result['enhanced'], cmap='gray')
        ax.set_title(f"TAPLA {i+1}\n"
                    f"EME: {result['eme']:.1f} | TEN: {result['ten']:.0f}\n"
                    f"Time: {result['time']:.3f}s", 
                    fontsize=9)
        ax.axis('off')
        
        # Histogram
        ax = plt.subplot(3, n_params + 1, n_params + 3 + i)
        ax.hist(result['enhanced'].ravel(), 256, [0, 256], 
                color=f'C{i}', alpha=0.7)
        ax.set_title(f"Histogram {i+1}", fontsize=9)
        ax.grid(True, alpha=0.3)
        
        # Parameters
        ax = plt.subplot(3, n_params + 1, 2*n_params + 4 + i)
        ax.axis('off')
        param_text = (f"c={result['params']['c']}\n"
                     f"γ={result['params']['gamma']}\n"
                     f"k={result['params']['k']}\n"
                     f"k₁={result['params']['k1']}\n"
                     f"w={result['params']['window_size']}")
        ax.text(0.5, 0.5, param_text, fontsize=9, ha='center', va='center',
                bbox=dict(boxstyle='round', facecolor=f'C{i}', alpha=0.3))
    
    plt.tight_layout()
    plt.show()
    
    return results


# =============================================================================
# EXAMPLE USAGE IN JUPYTER NOTEBOOK
# =============================================================================

print("""
╔══════════════════════════════════════════════════════════════════════╗
║       TAPLA Image Enhancement - Jupyter Notebook Version             ║
║  Threshold-based Adaptive Power-Law Applications                     ║
╚══════════════════════════════════════════════════════════════════════╝

Quick Start Examples:

1. BATCH PROCESSING (1000 images):
   ════════════════════════════════════
   # Process entire folder
   results = batch_enhance_images(
       input_folder='./input_images',
       output_folder='./enhanced_images',
       c=1.5, gamma=0.56, k=2.0, k1=0.2,
       color=True  # Set False for grayscale
   )
   
   # Preview results
   preview_batch_results('./enhanced_images', num_samples=6)
   
   # Compare original vs enhanced
   compare_batch_results('./input_images', './enhanced_images', num_samples=3)

2. Upload and enhance single image (Google Colab):
   ════════════════════════════════════
   filename = upload_image_jupyter()
   
   # For COLOR image:
   img = load_image(filename, color=True)
   enhanced, metrics = run_tapla_experiment(img, c=1.5, gamma=0.56, k=2.0, k1=0.2)

2. Use sample images (no upload needed):
   ════════════════════════════════════
   # Create test dataset
   create_sample_dataset('./test_images', num_images=10)
   
   # Process all images
   results = batch_enhance_images('./test_images', './output', color=True)

3. Compare multiple parameter sets:
   ════════════════════════════════════
   img = load_image('image.jpg', color=True)
   params_list = [
       {'c': 1.0, 'gamma': 2.5, 'k': 2.0, 'k1': 0.06, 'window_size': 11},
       {'c': 1.0, 'gamma': 1.1, 'k': 4.0, 'k1': 0.06, 'window_size': 11},
       {'c': 2.0, 'gamma': 1.5, 'k': -1.0, 'k1': 0.1, 'window_size': 11}
   ]
   results = compare_multiple_params(img, params_list)

BATCH PROCESSING TIPS:
══════════════════════
• Supports: .jpg, .jpeg, .png, .bmp
• Auto-saves metrics to CSV
• Progress bar shows real-time status
• Handles errors gracefully
• Creates output folder automatically

Parameter Guide:
   c      : brightness (typically 1.0-2.0)
   gamma  : contrast (>1 increase, <1 decrease)
   k      : sharpness (positive) or smoothness (negative)
   k1     : threshold control [0, 1]
   
Ready to use! 🚀
""")


╔══════════════════════════════════════════════════════════════════════╗
║       TAPLA Image Enhancement - Jupyter Notebook Version             ║
║  Threshold-based Adaptive Power-Law Applications                     ║
╚══════════════════════════════════════════════════════════════════════╝

Quick Start Examples:

1. BATCH PROCESSING (1000 images):
   ════════════════════════════════════
   # Process entire folder
   results = batch_enhance_images(
       input_folder='./input_images',
       output_folder='./enhanced_images',
       c=1.5, gamma=0.56, k=2.0, k1=0.2,
       color=True  # Set False for grayscale
   )

   # Preview results
   preview_batch_results('./enhanced_images', num_samples=6)

   # Compare original vs enhanced
   compare_batch_results('./input_images', './enhanced_images', num_samples=3)

2. Upload and enhance single image (Google Colab):
   ════════════════════════════════════
   filename = upload_image_jupyter()

   # For COLOR image:
   img = load_image(filena

In [14]:
results = batch_enhance_images(
       input_folder=r'D:\\Huce\\XLA\\LoLI-Street Dataset\\Test_50img_LL',
       output_folder=r"D:\\Huce\\XLA\\LoLI-Street Dataset\\Results_40Img_LL",
       c=1.5, gamma=0.56, k=2.0, k1=0.18,
       color=True  # Set False for grayscale
   )

BATCH TAPLA IMAGE ENHANCEMENT

📁 Input folder:  D:\\Huce\\XLA\\LoLI-Street Dataset\\Test_50img_LL
📁 Output folder: D:\\Huce\\XLA\\LoLI-Street Dataset\\Results_40Img_LL
🖼️  Found 80 images

⚙️  Parameters: c=1.5, γ=0.56, k=2.0, k₁=0.18, w=11
🎨 Mode: Color (RGB)



Processing images:   0%|          | 0/80 [00:00<?, ?it/s]

🎨 Processing color image (R, G, B channels separately)...
   Channel R... ✓
   Channel G... ✓
   Channel B... ✓
🎨 Processing color image (R, G, B channels separately)...
   Channel R... ✓
   Channel G... ✓
   Channel B... ✓
🎨 Processing color image (R, G, B channels separately)...
   Channel R... ✓
   Channel G... ✓
   Channel B... ✓
🎨 Processing color image (R, G, B channels separately)...
   Channel R... ✓
   Channel G... ✓
   Channel B... ✓
🎨 Processing color image (R, G, B channels separately)...
   Channel R... ✓
   Channel G... ✓
   Channel B... ✓
🎨 Processing color image (R, G, B channels separately)...
   Channel R... ✓
   Channel G... ✓
   Channel B... ✓
🎨 Processing color image (R, G, B channels separately)...
   Channel R... ✓
   Channel G... ✓
   Channel B... ✓
🎨 Processing color image (R, G, B channels separately)...
   Channel R... ✓
   Channel G... ✓
   Channel B... ✓
🎨 Processing color image (R, G, B channels separately)...
   Channel R... ✓
   Channel G... ✓
   Channel